In [1]:
!python -m pip install --user --upgrade pip

!pip3 install lasio pandas==0.24.2 matplotlib==3.2.2 scipy==1.4.1 statsmodels==0.12.0 scikit-learn==0.23.1 --user

     |████████████████████████████████| 1.5MB 7.3MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'pip install --upgrade pip' command.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     |████████████████████████████████| 12.4 MB 6.0 MB/s eta 0:00:01
     |████████████████████████████████| 10.1 MB 49.6 MB/s eta 0:00:01
     |████████████████████████████████| 6.8 MB 38.5 MB/s eta 0:00:01
     |████████████████████████████████| 9.5 MB 41.6 MB/s eta 0:00:01
     |████████████████████████████████| 302 kB 54.1 MB/s eta 0:00:01
     |████████████████████████████████| 231 kB 57.7 MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
!pip3 install kfp --upgrade --user

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     |████████████████████████████████| 165 kB 8.0 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 54 kB 3.9 MB/s  eta 0:00:01
     |████████████████████████████████| 54 kB 3.2 MB/s  eta 0:00:01
     |████████████████████████████████| 82 kB 1.3 MB/s  eta 0:00:01
  Created wheel for kfp: filename=kfp-1.2.0-py3-none-any.whl size=229698 sha256=402990d87c6e5d98cd18d1682871857cd9ae24b703a2e3ed56429a2583b69eb5
  Stored in directory: /home/jovyan/.cache/pip/wheels/4f/45/50/6fb5f06208872028e99b604f97cbc51467797e9495bf49fef8
  Created wheel for docstring-parser: filename=docstring_parser-0.7.3-py3-none-any.whl size=19230 sha256=176d641639349e89d93904b2d4192731785ff12411c3549d56baa

In [17]:
#Check if the install was successful
!which dsl-compile

In [18]:
# Import Kubeflow SDK
import kfp
import kfp.dsl as dsl
import kfp.components as comp

In [19]:
# where the outputs are stored
out_dir = "/home/jovyan/Volve_ML/data/"
holdout = False

In [20]:
def train_tranform(data_path):
    
    import pickle
    import sys, subprocess;
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas==0.24.2'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'scikit-learn==0.23.1'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'ipython'])
    
    import numpy as np
    import pandas as pd
    from sklearn.preprocessing import PowerTransformer
    
    # Download the dataset and split into training and test data. 
    data = pd.read_csv("https://raw.githubusercontent.com/Josepholaidepetro/Volve_ML/main/data/train.csv")
    
    # Preprocess
    data.drop(['DEPTH', 'BS', 'RD', 'ROP', 'RM', 'DRHO'], axis=1, inplace=True)
    
    # If Nan, drop
    data.dropna(inplace=True)
    
    # transform the RT to logarithmic
    data['RT'] = np.log10(data['RT'])
    
    # perform a yeo-johnson transform of the train dataset
    ptrain = PowerTransformer(method='yeo-johnson')
    train_df_yj = ptrain.fit_transform(data.drop('DT', axis=1))
    train_df_yj_norm = pd.DataFrame(train_df_yj, columns=data.columns.drop('DT'))
    y_train = data['DT']
    
    #Save the train_data as a pickle file to be used by the predict component.
    with open(f'{data_path}/train_data', 'wb') as f:
        pickle.dump((train_df_yj_norm,  y_train), f)

In [21]:
train_tranform(out_dir)

In [22]:
def test_tranform(data_path):
    
    import pickle
    import sys, subprocess;
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas==0.24.2'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'scikit-learn==0.23.1'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'ipython'])
    
    import numpy as np
    import pandas as pd
    from sklearn.preprocessing import PowerTransformer
    
    # Download the dataset and split into training and test data. 
    data = pd.read_csv("https://raw.githubusercontent.com/Josepholaidepetro/Volve_ML/main/data/test.csv")
    
    # Preprocess
    data.drop(['DEPTH', 'BS', 'ROP', 'DRHO'], axis=1, inplace=True)
    
    # If Nan, drop
    data.dropna(inplace=True)
    
    # transform the RT to logarithmic
    data['RT'] = np.log10(data['RT'])
    
    # perform a yeo-johnson transform of the train dataset
    ptest = PowerTransformer(method='yeo-johnson')
    test_df_yj = ptest.fit_transform(data.drop('DT', axis=1))
    test_df_yj_norm = pd.DataFrame(test_df_yj, columns=data.columns.drop('DT'))
    y_test = data['DT']
    
    #Save the test_data as a pickle file to be used by the predict component.
    with open(f'{data_path}/test_data', 'wb') as f:
        pickle.dump((test_df_yj_norm,  y_test), f)

In [23]:
test_tranform(out_dir)

/home/jovyan/.local/lib/python3.6/site-packages/sklearn/preprocessing/_data.py:2995: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


In [24]:
def Outlier_removal(data_path):
    
    import pickle
    import sys, subprocess;
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas==0.24.2'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'scikit-learn==0.23.1'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'ipython'])
    
    from sklearn.svm import OneClassSVM
    import numpy as np
    import pandas as pd
    
    # Load and unpack the train_data
    with open(f'{data_path}/train_data','rb') as f:
        train_data = pickle.load(f)
    # Separate the train_df_yj_norm from y_train.
    train_df_yj_norm,  y_train = train_data

    # Method 4: One-class SVM
    svm = OneClassSVM(nu=0.13)
    yhat = svm.fit_predict(train_df_yj_norm)
    mask = yhat != -1
    train_df_svm = train_df_yj_norm[mask]
    y_train_svm = y_train[mask]

    # prepare train data for modelling
    X_train = train_df_svm.copy().drop(['label'], axis=1)
    y_train = y_train_svm.copy()
    train_df_svm['DT'] = y_train
    
    #Save the train_data as a pickle file to be used by the predict component.
    with open(f'{data_path}/train_data2', 'wb') as f:
        pickle.dump((X_train,  y_train), f)
        
    #Save the train_data to be used for splitting as a pickle file to be used by the predict component.
    with open(f'{data_path}/train_split', 'wb') as f:
        pickle.dump((train_df_svm), f)
    
    print('Outlier removed successfully!')

In [25]:
Outlier_removal(out_dir)

Outlier removed successfully!


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [26]:
def valid_tranform(data_path):
    
    import pickle
    import sys, subprocess;
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas==0.24.2'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'scikit-learn==0.23.1'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'ipython'])
    
    import numpy as np
    import pandas as pd
    from sklearn.preprocessing import PowerTransformer
    
    # Load and unpack the train_data
    with open(f'{data_path}/train_split','rb') as f:
        train_split = pickle.load(f)
    X_train = train_split
    
    # Separate the data into X_train and y_train.
    X_train2 = X_train[X_train['label'] < 0].drop(['label', 'DT'], axis=1)
    y_train2 = X_train[X_train['label'] < 0]['DT']
    
    # Separate the data into X_valid and y_valid.
    X_valid = X_train[X_train['label'] > 0].drop(['label', 'DT'], axis=1)
    y_valid = X_train[X_train['label'] > 0]['DT']
    
    #Save the train_data and valid_data as a pickle file to be used by the predict component.
    with open(f'{data_path}/train_data3', 'wb') as f:
        pickle.dump((X_valid,  y_valid), f)
        
    #Save the train_data and valid_data as a pickle file to be used by the predict component.
    with open(f'{data_path}/train_data4', 'wb') as f:
        pickle.dump((X_train2,  y_train2), f)

In [27]:
valid_tranform(out_dir)

In [28]:
def model_building(data_path, holdout):
    
    import pickle
    import sys, subprocess;
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas==0.24.2'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'scikit-learn==0.23.1'])
    import numpy as np
    import pandas as pd
    from sklearn.ensemble import ExtraTreesRegressor
    
    # Load and unpack the full train_data2
    with open(f'{data_path}/train_data2','rb') as f:
        train_data2 = pickle.load(f)
    # Separate the X_train from y_train.
    X_train,  y_train = train_data2
    
    # Load and unpack the train_data4
    with open(f'{data_path}/train_data4','rb') as f:
        train_data4 = pickle.load(f)
    # Separate the X_train from y_train.
    X_train4,  y_train4 = train_data4
    
    # Load and unpack the valid_data
    with open(f'{data_path}/train_data3','rb') as f:
        train_data3 = pickle.load(f)
    # Separate the train_df_yj_norm from y_train.
    X_valid,  y_valid = train_data3
    
    # Load and unpack the test_data
    with open(f'{data_path}/test_data','rb') as f:
        test_data = pickle.load(f)
    # Separate the train_df_yj_norm from y_train.
    X_test, y_test = test_data
    
    # Define the model.
    model = ExtraTreesRegressor(n_estimators=800, max_depth=6, random_state=21)
    
    if holdout is False:
        # Run a training job
        model.fit(X_train, y_train)
    else:
        # Run a training job
        model.fit(X_train4, y_train4)

    
    #Save the model to the designated 
    filename = f'{data_path}/modelo.pkl'
    pickle.dump(model, open(filename, 'wb'))
    
    #Save holdout
    with open(f'{data_path}/holdout', 'wb') as f:
        pickle.dump((holdout), f)

In [29]:
regressor = model_building(out_dir, holdout=False)

In [30]:
def model_evaluate(data_path):
    
    import pickle
    import sys, subprocess;
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas==0.24.2'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'scikit-learn==0.23.1'])
    import numpy as np
    import pandas as pd
    from sklearn.ensemble import ExtraTreesRegressor
    
    file = f'{data_path}/holdout'
    holdout = pickle.load(open(file, 'rb'))
    
    # Load the saved model
    filename = f'{data_path}/modelo.pkl'
    model = pickle.load(open(filename, 'rb'))

    # Load and unpack the test_data
    with open(f'{data_path}/test_data','rb') as f:
        test_data = pickle.load(f)
    # Separate the X_test from y_test.
    X_test, y_test = test_data
    
    
    # Load and unpack the valid_data
    with open(f'{data_path}/train_data3','rb') as f:
        train_data3 = pickle.load(f)
    # Separate the train_df_yj_norm from y_train.
    X_valid,  y_valid = train_data3
    
    if holdout is False:
        #Evaluate the model and print the results
        score = model.score(X_test, y_test)
        print("R-squared of Well 3: {}".format(score))
    else:
        score = model.score(X_valid,y_valid)
        print("R-squared of Well 3: {}".format(score))


In [31]:
model_evaluate(out_dir)

R-squared of Well 3: 0.8683283360096595


In [32]:
def predict(data_path):
    
    import pickle
    import sys, subprocess;
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas==0.24.2'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'scikit-learn==0.23.1'])
    import numpy as np
    import pandas as pd
    from sklearn.ensemble import ExtraTreesRegressor
    
    # Load the holdout variable
    file = f'{data_path}/holdout'
    holdout = pickle.load(open(file, 'rb'))
    
    # Load the saved model
    filename = f'{data_path}/modelo.pkl'
    regressor = pickle.load(open(filename, 'rb'))

    if holdout is False:
        # Load and unpack the test_data
        with open(f'{data_path}/test_data','rb') as f:
            test_data = pickle.load(f)
        # Separate the X_test from y_test.
        test_df_yj_norm,  y_test = test_data

        # make predictions.
        y_pred = regressor.predict(test_df_yj_norm)
    else:
        # Load and unpack the test_data
        with open(f'{data_path}/test_data','rb') as f:
            test_data = pickle.load(f)
        # Separate the X_test from y_test.
        test_df_yj_norm,  y_test = test_data

        # make predictions.
        y_pred = regressor.predict(test_df_yj_norm)
        
    
    with open(f'{data_path}/result1', 'wb') as f:
        pickle.dump(y_pred, f)
        
    with open(f'{data_path}/result.txt', 'w') as result:
        result.write(" Prediction: {}, Actual: {} ".format(y_pred,y_test))
    
    print('Prediction has be saved successfully!')

In [33]:
# Create train and predict lightweight components.
train_tranform_op = comp.func_to_container_op(train_tranform , base_image = "tensorflow/tensorflow:latest-gpu-py3")
test_tranform_op = comp.func_to_container_op(test_tranform , base_image = "tensorflow/tensorflow:latest-gpu-py3")
valid_tranform_op = comp.func_to_container_op(valid_tranform , base_image = "tensorflow/tensorflow:latest-gpu-py3")
Outlier_removal_op = comp.func_to_container_op(Outlier_removal , base_image = "tensorflow/tensorflow:latest-gpu-py3")
train_op = comp.func_to_container_op(model_building , base_image = "tensorflow/tensorflow:latest-gpu-py3")
valid_op = comp.func_to_container_op(model_evaluate , base_image = "tensorflow/tensorflow:latest-gpu-py3")
predict_op = comp.func_to_container_op(predict , base_image = "tensorflow/tensorflow:latest-gpu-py3")

## Build Kubeflow Pipeline

In [34]:
#Create a client to enable communication with the Pipelines API server.
client = kfp.Client()

In [39]:
# Define the pipeline
@dsl.pipeline(
   name='Sonic Log Prediction Pipeline',
   description='An ML pipeline that performs Sonic Log Prediction prediction.'
)

# Define parameters to be fed into pipeline
def sonic_container_pipeline(
    data_path: str,
    holdout: bool
    ):
    
    # Define volume to share data between components.
    vop = dsl.VolumeOp(
    name="create_volume",
    resource_name="data-volume", 
    size="1Gi", 
    modes=dsl.VOLUME_MODE_RWO) # read write one
    
    # Create deploy component.
    train_tranform_container = train_tranform_op(data_path) \
                                    .add_pvolumes({data_path: vop.volume})

    # Create data transformation component.
    test_tranform_container = test_tranform_op(data_path) \
                                    .add_pvolumes({data_path: train_tranform_container.pvolume})
    
    # Create data transformation component.
    valid_tranform_container = valid_tranform_op(data_path) \
                                    .add_pvolumes({data_path: test_tranform_container.pvolume})
    
    # Create data transformation component.
    Outlier_removal_container = Outlier_removal_op(data_path) \
                                    .add_pvolumes({data_path: valid_tranform_container.pvolume})
    
    # Create model training component.
    train_container = train_op(data_path, holdout) \
                                    .add_pvolumes({data_path: Outlier_removal_container.pvolume})
    
    # Create model validation component.
    valid_container = valid_op(data_path) \
                                    .add_pvolumes({data_path: train_container.pvolume})
    
    # Create model training component.
    predict_container = predict_op(data_path) \
                                    .add_pvolumes({data_path: valid_container.pvolume})
    
    
    # Print the result of the prediction
    churn_result_container = dsl.ContainerOp(
        name="print_prediction",
        image='library/bash:4.4.23', # Image tag for the Docker container to be used.
        pvolumes={data_path: predict_container.pvolume}, # the name displayed for the component execution during runtime.
        arguments=['cat', f'{data_path}/result.txt']
    )

## Run the Pipeline

In [40]:
pipeline_func = sonic_container_pipeline

In [41]:
DATA_PATH = out_dir # mount your filesystems or directory

In [42]:
experiment_name = 'dt_prediction_kubeflow'
run_name = pipeline_func.__name__ + ' run'

arguments = {"data_path":DATA_PATH}

# Compile pipeline to generate compressed YAML definition of the pipeline.
kfp.compiler.Compiler().compile(pipeline_func, '{}.zip'.format(experiment_name))

# Submit pipeline directly from pipeline function
run_result = client.create_run_from_pipeline_func(pipeline_func, 
                                                  experiment_name=experiment_name, 
                                                  run_name=run_name, 
                                                  arguments=arguments)

/home/jovyan/.local/lib/python3.6/site-packages/kfp/dsl/_container_op.py:1039: FutureWarning: Please create reusable components instead of constructing ContainerOp instances directly. Reusable components are shareable, portable and have compatibility and support guarantees. Please see the documentation: https://www.kubeflow.org/docs/pipelines/sdk/component-development/#writing-your-component-definition-file The components can be created manually (or, in case of python, using kfp.components.create_component_from_func or func_to_container_op) and then loaded using kfp.components.load_component_from_file, load_component_from_uri or load_component_from_text: https://kubeflow-pipelines.readthedocs.io/en/stable/source/kfp.components.html#kfp.components.load_component_from_file
  category=FutureWarning,


http://localhost:8888/pipeline#/experiments/details/644a33c5-3114-4b3c-b6de-3b0b6467eafa

http://localhost:8888/pipeline#/runs/details/2aadb518-ead4-4035-90ea-011a44bfe16d